In [2]:
import requests
import json
import pandas as pd

def fetch_drugs():
    url = "https://api.fda.gov/drug/label.json"
    params = {'limit': 5}  # Limite à 5 pour l'exemple
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return pd.json_normalize(response.json()['results'])
    else:
        print("Erreur API:", response.status_code)
        return pd.DataFrame()

# Récupérer les données
drugs_df = fetch_drugs()

# Afficher pour vérifier
print(drugs_df.head())

  effective_time                                inactive_ingredient  \
0       20210902                     [INACTIVE INGREDIENTS Sucrose]   
1       20150109  [INGREDIENTS: TALC, POLYMETHYL METHACRYLATE, V...   
2       20250102  [Inactive ingredients pareth 25-9, purified wa...   
3       20230802             [INACTIVE INGREDIENTS Sucrose/Lactose]   
4       20230905                                                NaN   

                                             purpose  \
0  [USES USES: Temporary Relief - Acne, Boils* * ...   
1                                [Purpose Sunscreen]   
2                     [Purpose First aid Antiseptic]   
3         [USES To relieve the symptoms of itching.]   
4                                                NaN   

                       keep_out_of_reach_of_children  \
0  [Keep this and all medication out of reach of ...   
1  [Keep out of reach of children If product is s...   
2  [Keep out of reach of children If swallowed, g...   
3  [KEEP OUT

In [3]:
import json
import random
from faker import Faker
from datetime import datetime, timedelta

fake = Faker('fr_FR')

# Map diagnostic => symptômes typiques
diagnosisSymptomsMap = {
    "grippe": ["fièvre", "toux", "fatigue", "courbatures"],
    "COVID-19": ["fièvre", "toux sèche", "perte d’odorat", "fatigue"],
    "migraine": ["maux de tête", "nausées", "sensibilité à la lumière"],
    "angine": ["gorge irritée", "fièvre", "douleur en avalant"],
    "varicelle": ["éruption cutanée", "fièvre", "démangeaisons"],
    "asthme": ["essoufflement", "sifflements", "toux"],
    "allergie": ["éternuements", "yeux rouges", "démangeaisons"],
    "gastro-entérite": ["nausées", "vomissements", "diarrhée", "crampes"],
    "hypertension": ["maux de tête", "vertiges", "saignement de nez"]
}

def generatePatient(i):
    diagnosis = random.choice(list(diagnosisSymptomsMap.keys()))
    symptomList = diagnosisSymptomsMap[diagnosis]
    k = min(len(symptomList), random.randint(2, 4))
    symptoms = random.sample(symptomList, k=k)

    
    patient = {
        "patient_id": f"P{str(i).zfill(3)}",
        "age": random.randint(18, 90),
        "gender": random.choice(["M", "F"]),
        "symptoms": symptoms,
        "diagnosis": diagnosis,
        "timestamp": (datetime.now() - timedelta(days=random.randint(0, 365))).isoformat(),
        "notes": fake.sentence() if random.random() > 0.3 else None
    }
    return patient

# Générer 50 patients
patients = [generatePatient(i) for i in range(1, 51)]

# Écrire dans un fichier JSON
with open("patients.json", "w", encoding="utf-8") as f:
    json.dump(patients, f, indent=2, ensure_ascii=False)

print("✅ patients.json généré avec succès (50 entrées)")


✅ patients.json généré avec succès (50 entrées)


In [4]:
import psycopg2
import random
from faker import Faker
from datetime import datetime, timedelta

fake = Faker('fr_FR')

# Mapping symptômes typiques par maladie
diagnosisSymptomsMap = {
    "grippe": ["fièvre", "toux", "fatigue", "courbatures"],
    "COVID-19": ["fièvre", "toux sèche", "perte d’odorat", "fatigue"],
    "migraine": ["maux de tête", "nausées", "sensibilité à la lumière"],
    "angine": ["gorge irritée", "fièvre", "douleur en avalant"],
    "asthme": ["essoufflement", "sifflements", "toux"],
    "allergie": ["éternuements", "yeux rouges", "démangeaisons"],
    "gastro-entérite": ["nausées", "vomissements", "diarrhée", "crampes"],
    "hypertension": ["maux de tête", "vertiges", "saignement de nez"],
    "varicelle": ["fièvre", "éruption cutanée", "démangeaisons"]
}

statuses = ["terminée", "en attente", "en cours"]
medicalCenters = ["CHU Lyon", "Clinique Pasteur", "Hôpital Saint-Louis", "Centre Médical Dijon"]

# Connexion PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="medical_db",
    user="med_user",
    password="med_pass"
)
cursor = conn.cursor()

# Génération et insertion
for i in range(50):
    patientCode = f"P{str(random.randint(1, 999)).zfill(3)}"
    consultationDate = datetime.now() - timedelta(days=random.randint(0, 365))
    diagnosis = random.choice(list(diagnosisSymptomsMap.keys()))
    symptomList = diagnosisSymptomsMap[diagnosis]
    symptoms = random.sample(symptomList, k=min(len(symptomList), random.randint(2, 4)))
    doctor = f"Dr {fake.last_name()}"
    center = random.choice(medicalCenters)
    status = random.choice(statuses)

    cursor.execute("""
        INSERT INTO consultations 
        (patient_code, consultation_date, symptoms, diagnosis, doctor_name, medical_center, status)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (
        patientCode,
        consultationDate,
        symptoms,
        diagnosis,
        doctor,
        center,
        status
    ))

conn.commit()
cursor.close()
conn.close()

print("✅ 50 consultations insérées avec succès dans la base PostgreSQL.")


✅ 50 consultations insérées avec succès dans la base PostgreSQL.


In [5]:
import requests
import pandas as pd

url = "https://api.fda.gov/drug/label.json"
params = {"limit": 50}  # On prend 10 entrées pour commencer

response = requests.get(url, params=params)
data = response.json()

# Normaliser le JSON pour extraire les colonnes utiles
df = pd.json_normalize(data["results"])

# Exemple : extrait nom médicament + usage
meds_df = df[["id", "openfda.brand_name", "purpose", "indications_and_usage"]]

print(meds_df.head())


                                     id openfda.brand_name  \
0  ca7bbcc8-2354-375c-e053-2995a90a72a0          [SILICEA]   
1  f229e866-5775-4e42-a316-8480dd92fec6                NaN   
2  b8f5797b-73e1-4201-b824-2cdd50c90497         [Betadine]   
3  01f4b0f6-94df-fd91-e063-6394a90a7997                NaN   
4  f8ce57b8-ebf7-4dc1-96ec-c8fbc41c17ff                NaN   

                                             purpose  \
0  [USES USES: Temporary Relief - Acne, Boils* * ...   
1                                [Purpose Sunscreen]   
2                     [Purpose First aid Antiseptic]   
3         [USES To relieve the symptoms of itching.]   
4                                                NaN   

                               indications_and_usage  
0  [INDICATIONS Condition listed above or as dire...  
1  [Uses Multi-purpose mineral powder provides br...  
2  [Uses First aid to help prevent infection in m...  
3        [INDICATIONS Indications: MEZEREUM Itching]  
4  [INDICATIONS